In [5]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import gkey
import json
import numpy as np

In [6]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [7]:
Base.prepare(engine, reflect=True)

In [8]:
session = Session(engine)

In [9]:
#create dataframe
brew_df = pd.read_sql_query("SELECT id, name, street, city, state, zipcode, latitude, longitude FROM breweries", con=engine)
brew_df

,id,name,street,city,state,zipcode,latitude,longitude
0,5016,Bark Brewing Company,3021 Spring Garden St,Greensboro,North Carolina,27403,36.0620103554184,-79.8436043406271
1,5033,Blowing Rock Brewing Company,152 Sunset Dr,Blowing Rock,North Carolina,28605,36.1327196,-81.677229
2,5035,Blue Ghost Brewing Company,125 Underwood Rd,Fletcher,North Carolina,28732,35.4368388,-82.530580304217
3,5067,Bull Durham Beer Co,409 Blackwell St,Durham,North Carolina,27701,35.9915653,-78.90481567784
4,5090,Dirtbag Ales Brewery and Taproom,3623 Legion Rd,Hope Mills,North Carolina,28348,34.9939752576824,-78.9220969691311
...,...,...,...,...,...,...,...,...
261,5273,Thirsty Monk Brewery,92 Patton Ave,Asheville,North Carolina,28801,35.59410425,-82.5553147520814
262,5283,Triskelion Brewing Company,340 7th Avenue East,Hendersonville,North Carolina,28792,35.3206001746298,-82.4578969903109
263,5293,Valley River Brewery & Eatery,71 Tennessee St,Murphy,North Carolina,28906,35.0889040369639,-84.0352081706441
264,5304,White Street Brewing Company,218 S White St,Wake Forest,North Carolina,27587,35.9774896,-78.5100617


In [10]:
#set base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
place_url = "https://maps.googleapis.com/maps/api/place/details/json"

In [11]:
#create columns for new data from api call
brew_df['rating']=''
brew_df['user_ratings_total']=''
brew_df

,id,name,street,city,state,zipcode,latitude,longitude,rating,user_ratings_total
0,5016,Bark Brewing Company,3021 Spring Garden St,Greensboro,North Carolina,27403,36.0620103554184,-79.8436043406271,,
1,5033,Blowing Rock Brewing Company,152 Sunset Dr,Blowing Rock,North Carolina,28605,36.1327196,-81.677229,,
2,5035,Blue Ghost Brewing Company,125 Underwood Rd,Fletcher,North Carolina,28732,35.4368388,-82.530580304217,,
3,5067,Bull Durham Beer Co,409 Blackwell St,Durham,North Carolina,27701,35.9915653,-78.90481567784,,
4,5090,Dirtbag Ales Brewery and Taproom,3623 Legion Rd,Hope Mills,North Carolina,28348,34.9939752576824,-78.9220969691311,,
...,...,...,...,...,...,...,...,...,...,...
261,5273,Thirsty Monk Brewery,92 Patton Ave,Asheville,North Carolina,28801,35.59410425,-82.5553147520814,,
262,5283,Triskelion Brewing Company,340 7th Avenue East,Hendersonville,North Carolina,28792,35.3206001746298,-82.4578969903109,,
263,5293,Valley River Brewery & Eatery,71 Tennessee St,Murphy,North Carolina,28906,35.0889040369639,-84.0352081706441,,
264,5304,White Street Brewing Company,218 S White St,Wake Forest,North Carolina,27587,35.9774896,-78.5100617,,


In [12]:
#hit google places api

for index, row in brew_df.iterrows():
    params = {
        "key": gkey,
        "query": f"brewery",
        "location":f"{row.latitude},{row.longitude}"
    }
    
    response = requests.get(base_url, params=params)
    place = response.json()['results'][0]
        
    try:
        brew_df.at[index, 'rating'] = place['rating']
        brew_df.at[index, 'user_ratings_total'] = place['user_ratings_total']

    except:
        pass

In [13]:
brew_df

,id,name,street,city,state,zipcode,latitude,longitude,rating,user_ratings_total
0,5016,Bark Brewing Company,3021 Spring Garden St,Greensboro,North Carolina,27403,36.0620103554184,-79.8436043406271,3.5,4
1,5033,Blowing Rock Brewing Company,152 Sunset Dr,Blowing Rock,North Carolina,28605,36.1327196,-81.677229,4.2,241
2,5035,Blue Ghost Brewing Company,125 Underwood Rd,Fletcher,North Carolina,28732,35.4368388,-82.530580304217,4.7,206
3,5067,Bull Durham Beer Co,409 Blackwell St,Durham,North Carolina,27701,35.9915653,-78.90481567784,4.2,6
4,5090,Dirtbag Ales Brewery and Taproom,3623 Legion Rd,Hope Mills,North Carolina,28348,34.9939752576824,-78.9220969691311,4.7,160
...,...,...,...,...,...,...,...,...,...,...
261,5273,Thirsty Monk Brewery,92 Patton Ave,Asheville,North Carolina,28801,35.59410425,-82.5553147520814,4.6,293
262,5283,Triskelion Brewing Company,340 7th Avenue East,Hendersonville,North Carolina,28792,35.3206001746298,-82.4578969903109,4.7,166
263,5293,Valley River Brewery & Eatery,71 Tennessee St,Murphy,North Carolina,28906,35.0889040369639,-84.0352081706441,4.4,696
264,5304,White Street Brewing Company,218 S White St,Wake Forest,North Carolina,27587,35.9774896,-78.5100617,4.6,185


In [25]:
# load in NC brewery data CSV
nc_commerce_csv = "NC-Breweries_3-24-2020_clean-data.csv"

# make into a data frame
nc_breweries = pd.read_csv(nc_commerce_csv)
nc_breweries

,Company Name,street,City,State,Zip Code,Zip Plus,Contact First Name,Contact Last Name,Contact Title,Phone,...,NAICS2,NAICS3,NAICS4,NAICS5,Business Type,County,Employee,Annual Sales,URL,Year Established
0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,2529916958,...,NaN,NaN,NaN,NaN,Private,Wilson,14,10453000.0,217BREWWORKS.COM,2016
1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,NaN,NaN,NaN,8282259782,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,NaN,2018
2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,NaN,NaN,NaN,8284123212,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,NaN,2017
3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,NaN,NaN,NaN,8283212006,...,NaN,NaN,NaN,NaN,Private,Cherokee,14,17916000.0,NaN,2018
4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,3368350103,...,NaN,NaN,NaN,NaN,Private,Surry,12,16648000.0,ANGRYTROLLBREWING.COM,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Woodfin ABC,142 Weaverville Rd,Asheville,NC,28804,1238.0,Tammy,Ballew,Manager,8286589336,...,NaN,NaN,NaN,NaN,Private,Buncombe,5,2240000.0,WOODFINABC.NET,2007
909,Worth the Wine Co,1076 Weddington Woods St NW,Concord,NC,28027,8143.0,Lindsay G,Hitt,Owner,7040001111,...,NaN,NaN,NaN,NaN,Private,Cabarrus,2,465000.0,NaN,2016
910,Ws Wine Market LLC,1231 Creekshire Way,Winston Salem,NC,27103,3057.0,Elizabeth A,Binder,Manager,3368936950,...,NaN,NaN,NaN,NaN,Private,Forsyth,1,482000.0,NaN,2017
911,Youngsville ABC Store,101 N College St,Youngsville,NC,27596,NaN,Laurie,Leedy,Manager,9195563590,...,NaN,NaN,NaN,NaN,Private,Franklin,3,654000.0,NaN,2002


In [29]:
# merge datasets
brew_merge = pd.merge(brew_df, nc_breweries, on="street", how="left")
brew_merge

,id,name,street,city,state,zipcode,latitude,longitude,rating,user_ratings_total,...,NAICS2,NAICS3,NAICS4,NAICS5,Business Type,County,Employee,Annual Sales,URL,Year Established
0,5016,Bark Brewing Company,3021 Spring Garden St,Greensboro,North Carolina,27403,36.0620103554184,-79.8436043406271,3.5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5033,Blowing Rock Brewing Company,152 Sunset Dr,Blowing Rock,North Carolina,28605,36.1327196,-81.677229,4.2,241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5035,Blue Ghost Brewing Company,125 Underwood Rd,Fletcher,North Carolina,28732,35.4368388,-82.530580304217,4.7,206,...,NaN,NaN,NaN,NaN,Private,Henderson,14.0,20693000.0,BLUEGHOSTBREWING.COM,2016.0
3,5067,Bull Durham Beer Co,409 Blackwell St,Durham,North Carolina,27701,35.9915653,-78.90481567784,4.2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5090,Dirtbag Ales Brewery and Taproom,3623 Legion Rd,Hope Mills,North Carolina,28348,34.9939752576824,-78.9220969691311,4.7,160,...,312120.0,NaN,NaN,NaN,Private,Cumberland,4.0,946000.0,DIRTBAGALES.COM,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,5273,Thirsty Monk Brewery,92 Patton Ave,Asheville,North Carolina,28801,35.59410425,-82.5553147520814,4.6,293,...,722511.0,722410.0,NaN,NaN,Private,Buncombe,15.0,22507000.0,MONKPUB.COM,2008.0
263,5283,Triskelion Brewing Company,340 7th Avenue East,Hendersonville,North Carolina,28792,35.3206001746298,-82.4578969903109,4.7,166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,5293,Valley River Brewery & Eatery,71 Tennessee St,Murphy,North Carolina,28906,35.0889040369639,-84.0352081706441,4.4,696,...,NaN,NaN,NaN,NaN,Private,Cherokee,6.0,7678000.0,VALLEYRIVERBREWERY.COM,2015.0
265,5304,White Street Brewing Company,218 S White St,Wake Forest,North Carolina,27587,35.9774896,-78.5100617,4.6,185,...,445310.0,NaN,NaN,NaN,Private,Wake,4.0,1761000.0,WHITESTREETBREWING.COM,2012.0
